In [ ]:
# 🚀 Crypto Dashboard API Automation
# Author: Your Name
# GitHub: https://github.com/YourUsername/Crypto-Dashboard-API-Automation

from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep
from IPython.display import clear_output
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

# ==============================
# CONFIGURATION
# ==============================

# 🔑 API Key Setup (User should add their own key here or via environment variable)
API_KEY = os.getenv("CMC_API_KEY", "YOUR_API_KEY_HERE")  
# Example: in terminal → set CMC_API_KEY=your_key

# API URL
URL = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"

# Save location
SAVE_PATH = r"S:\Crypto\API.csv"

# ==============================
# FUNCTIONS
# ==============================

def api_runner():
    """Fetch data from CoinMarketCap API and save to CSV."""
    global df
    parameters = {"start": "1", "limit": "15", "convert": "USD"}
    headers = {"Accepts": "application/json", "X-CMC_PRO_API_KEY": API_KEY}

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(URL, params=parameters)
        data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print("API Error:", e)
        return

    df2 = pd.json_normalize(data["data"])
    df2["timestamp"] = pd.to_datetime("now")

    if os.path.isfile(SAVE_PATH):
        df2.to_csv(SAVE_PATH, mode="a", header=False, index=False)
    else:
        df2.to_csv(SAVE_PATH, index=False)

    print("API Runner completed")

def live_visuals():
    """Generate dashboard and save snapshot to PDF."""
    if not os.path.exists(SAVE_PATH):
        print("⚠️ CSV file not found. Run api_runner() first.")
        return

    df = pd.read_csv(SAVE_PATH)
    if "timestamp" not in df.columns and "Timestamp" in df.columns:
        df.rename(columns={"Timestamp": "timestamp"}, inplace=True)

    df["timestamp"] = pd.to_datetime(df["timestamp"])

    latest = df[df["timestamp"] == df["timestamp"].max()]

    # Create dashboard
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))

    sns.barplot(x="name", y="quote.USD.price", data=latest, ax=axes[0, 0])
    axes[0, 0].set_title("Latest Prices")

    btc_data = df[df["name"] == "Bitcoin"]
    sns.lineplot(x="timestamp", y="quote.USD.price", data=btc_data, ax=axes[0, 1])
    axes[0, 1].set_title("Bitcoin Price Trend")

    sns.barplot(x="name", y="quote.USD.percent_change_24h", data=latest, ax=axes[1, 0])
    axes[1, 0].set_title("24h % Change")

    top5 = latest.nlargest(5, "quote.USD.market_cap")
    sns.barplot(x="name", y="quote.USD.market_cap", data=top5, ax=axes[1, 1])
    axes[1, 1].set_title("Top 5 Coins by Market Cap")

    plt.tight_layout()
    clear_output(wait=True)
    plt.show()

    # Save snapshot to PDF
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    pdf_path = rf"S:\Crypto\Crypto_Dashboard_{timestamp}.pdf"

    with PdfPages(pdf_path) as pdf:
        pdf.savefig(fig)

    print(f"📄 Dashboard snapshot saved: {pdf_path}")

# ==============================
# MAIN LOOP (Optional)
# ==============================

# Run API once and update visuals
api_runner()
live_visuals()
